In [1]:
import os,sys,inspect
import json
import torch
currentdir = os.path.dirname(os.getcwd())
print(currentdir)
# parent_dir = os.path.abspath(os.path.join(currentdir, os.pardir))
root = os.path.dirname(currentdir)
print("root Folder Absoloute path: ", root)


import sys
sys.path.append(root)

import logging

logging.basicConfig(level=logging.INFO)

from IPython.display import Image

import __main__

__main__.__file__="main.py"

/home/hfaghihi/Framework/new/DomiKnowS/examples
root Folder Absoloute path:  /home/hfaghihi/Framework/new/DomiKnowS


In [2]:
with open("data/tracker.log", "r") as file:
    tracker = []
    for line in file:
        tracker.append(json.loads(line))
print(len(tracker))
print(tracker[0].keys())

54
dict_keys(['para_id', 'location_cand', 'states', 'loc_spans', 'boundaries', 'states_mapping', 'states_in_tokens', 'spans', 'roberta_tokens', 'states_annotation', 'roberta_bounds', 'participants', 'sentence_paragraph', 'sentence_texts', 'trips_annotation', 'actions', 'ltype', 'net_results'])


In [3]:
action_probs = torch.tensor([tracker[0]['net_results'][i]['action_probabilities'] for i in range(len(tracker[0]['sentence_texts']))]).squeeze(-2)
print(action_probs.shape)
print(action_probs.softmax(dim=-1))
print(len(tracker[0]['sentence_texts']))

torch.Size([6, 5, 4])
tensor([[[3.1940e-01, 4.0114e-01, 1.0121e-02, 2.6934e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01]],

        [[7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01]],

        [[5.7955e-01, 3.5761e-04, 3.9642e-01, 2.3672e-02],
         [4.5550e-01, 2.2812e-02, 2.1059e-02, 5.0063e-01],
         [2.4481e-01, 1.5927e-05, 5.0667e-01, 2.4850e-01],
         [5.7955e-01, 3.5761e-04, 3.9642e-01, 2.3672e-02],
         [5.7955e-01, 3.5761e-04, 3.9642e-01, 2.3672e-02]],

        [[5.8445e-01, 2.2438e-04, 4.1525e-01, 7.7910e-05],
         [4.7322e-01, 1.5209

In [4]:
import json
import torch

with open("data/train_propara_roberta_version_trips.json", "r") as file:
    data = json.load(file)
print(len(data))
updated_data = []
lines = 0
for item in tracker:
    lines += len(item["sentence_texts"])
    try:
        temp = item.copy()
        new_state = []
        location_types = []
        for state_id, states in enumerate(temp['states']):
            temp2 = ["-", ]
            
            ltemp = []
            if states[0] == "?":
                ltemp.append("Unknown")
            elif states[0] == "-":
                ltemp.append("-")
            else:
                ltemp.append("Known")
                
            prev_loc = states[0]
            for loc in states[1:]:
                if loc == "?":
                    ltemp.append("Unknown")
                elif loc == "-":
                    ltemp.append("-")
                else:
                    ltemp.append("Known")

                if loc == prev_loc:
                    temp2.append("No change")
                elif loc != "-" and prev_loc == "-":
                    temp2.append("Create")
                elif loc == "-" and prev_loc != "-":
                    temp2.append("Destroy")
                else:
                    temp2.append("Move")
                prev_loc = loc
            new_state.append(temp2)
            location_types.append(ltemp)
        temp['actions'] = new_state
        temp['ltype'] = location_types
        

#         new_annotation = []
#         for state in temp['states_annotation']:
#             temp2 = state
#             for loc in temp2:
#                 if loc[0] == "?":
#                     loc[2] = item['boundaries'][-1][1] + 1
#                     loc[3] = item['boundaries'][-1][1] + 1
#             new_annotation.append(temp2)

#         temp['new_annotation'] = new_annotation
    except:
        print(item)
        raise
    updated_data.append(temp)
    
for item in updated_data:
    steps = len(item['sentence_paragraph'])
    entities = len(item['participants'])
    probs = action_probs = torch.tensor([item['net_results'][i]['action_probabilities'] for i in range(len(item['sentence_texts']))]).squeeze(-2).softmax(dim=-1)
    item['action_probs'] = probs.tolist()
#     tprobs = torch.softmax(torch.randn(steps, entities, 4), dim=-1)
#     item['Taction_probs'] = tprobs.tolist()
    
with open("data/train.json", "w") as file:
    json.dump(updated_data, file)

391


In [5]:
with open("data/train.json", "r") as file:
    data = json.load(file)
print(len(data))

54


In [6]:
from regr.data.reader import RegrReader
import torch


class ProparaReader(RegrReader):
    def getProcedureIDval(self, item):
        return [item["para_id"]]

    def getEntitiesval(self, item):
        return [' '.join(item['participants'])]
        
    def getEntityval(self, item):
        return item["participants"]
    
    def getContextval(self, item):
        return [item['sentence_paragraph']]

    def getStepval(self, item):
        sentences = item["sentence_texts"]
        return  sentences
    
    def getActionval(self, item):
        return torch.tensor(item['action_probs'])
    
    def getbeforeval(self, item):
        b1s = []
        b2s = []
        for step in range(len(item["sentence_texts"])):
            b1 = torch.zeros(len(item["sentence_texts"]))
            b1[step] = 1
            for step1 in range(len(item["sentence_texts"])):
                b2 = torch.zeros(len(item["sentence_texts"]))
                b2[step1] = 1
                b1s.append(b1)
                b2s.append(b2)
        return torch.stack(b1s), torch.stack(b2s)

    def getbefore_trueval(self, item):
        num_steps = len(item["sentence_texts"])
        values = torch.zeros(num_steps * num_steps)
        for step in range(len(item["sentence_texts"])):
            for step1 in range(step + 1, len(item["sentence_texts"])):
                values[(step * num_steps) + step1] = 1
        return values
    
#     def getTActionval(self, item):
#         actions = []
#         for step, step_text in enumerate(item['sentence_texts']):
#             actions.append([])
#             for eid, entity in enumerate(item['participants']):
#                 actions[-1].append(item['Taction_probs'])

#         return torch.tensor(actions)


In [7]:
train_reader = ProparaReader("data/train.json")
sample = next(iter(train_reader))
print(sample['before'][0].shape)
print(sample['before'][1].shape)
print(sample['before_true'].shape)
print(len(sample['Step']))

torch.Size([36, 6])
torch.Size([36, 6])
torch.Size([36])
6


In [8]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL, nandL, V, exactL
from regr.graph import EnumConcept


Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept("procedure")
    
    context = Concept("context")
    entities = Concept("entities")
    
    (procedure_context, procedure_entities) = procedure.has_a(context, entities)
    
    entity = Concept('entity')
    (entity_rel, ) = entities.contains(entity)
    
    step = Concept("step")
    (context_step, ) = context.contains(step)
    
    before = Concept("before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    
    action = Concept(name='action')
    (action_step, action_entity) = action.has_a(step, entity)
    
    
    action_label = action(name="action_label", ConceptClass=EnumConcept, values=["nochange", "destroy", "create", "move"])
#     create = action(name="create")
#     destroy = action(name="destroy")
#     move = action(name="move")
#     nochange = action(name='nochange')
    
#     Tcreate = action(name="trips_create")
#     Tdestroy = action(name="trips_destroy")
#     Tmove = action(name="trips_move")
#     Tnochange = action('trips_none')

#     exactL(Tcreate, Tdestroy, Tmove, Tnochange)
#     exactL(create, destroy, move, nochange)
    
#     ifL(Tcreate, create)
#     ifL(Tdestroy, destroy)
#     ifL(Tmove, move)
#     ifL(Tnochange, nochange)
    
#     ifL(
#         destroy('x'), 
#         notL(
#             andL(
#                 existsL(
#                     andL(destroy('y'), existsL(before('y', path=('x', arg1, 'before', arg1)))),
#                     notL(
#                         existsL(
#                             andL(
#                                 create('z'), 
#                                 eqL(before('z'))
#                             )
#                         )
#                     )
#                 )
#             )
#         )
#     )
#   atMostL(1, ("x"), andL(entity, "y", create()))
    # No entity_step




Log file for dataNode is in: /home/hfaghihi/Framework/new/DomiKnowS/examples/ModularPropara/logs/datanode.log


main.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
  before_arg1,


In [9]:
from regr.sensor.pytorch.sensors import FunctionalSensor, JointSensor, ReaderSensor, FunctionalReaderSensor
from regr.sensor.pytorch.learners import ModuleLearner
from regr.sensor.pytorch.relation_sensors import CompositionCandidateSensor
from regr.sensor.pytorch.query_sensor import DataNodeReaderSensor

class JointFunctionalReaderSensor(JointSensor, FunctionalReaderSensor):
    pass


procedure['id'] = ReaderSensor(keyword='ProcedureID')
context['text'] = ReaderSensor(keyword='Context')
entities['text'] = ReaderSensor(keyword='Entities')

def make_procedure(arg1m, arg2m, data):
    total_procedures = len(arg1m) * len(arg2m)
    rel_links1 = torch.zeros(total_procedures, len(arg1m))
    rel_links2 = torch.zeros(total_procedures, len(arg2m))
    past1 = 0
    past2 = 0
    for i in range(total_procedures):
        rel_links1[i, past2: past2 + len(arg2m)] = 1
        past2 = past2 + len(arg2m)
        rel_links2[i, past1: past1 + len(arg1m)] = 1
        past1 = past1 + len(arg1m)

    return rel_links1, rel_links2
    
    
procedure[procedure_context.reversed, procedure_entities.reversed] = JointFunctionalReaderSensor(context['text'], entities['text'], keyword="ProcedureID", forward=make_procedure)

def read_initials(*prev, data):
    number = len(data)
    rel_links = torch.ones(number, 1)
    indexes = [i for i in range(number)]
    print(rel_links, data, indexes)
    return rel_links, data, indexes

entity[entity_rel, 'text', 'index'] = JointFunctionalReaderSensor(entities['text'], keyword='Entity', forward=read_initials)

step[context_step, 'text', 'index'] = JointFunctionalReaderSensor(context['text'], keyword='Step', forward=read_initials)

def make_actions(r1, r2, entities, steps):
    print(r1, r2)
    all_actions = len(steps) * len(entities)
    link1 = torch.zeros(all_actions, len(steps))
    link2 = torch.zeros(all_actions, len(entities))
    for i in range(len(steps)):
        link1[i*len(entities):(i+1)*len(entities),i] = 1

    for j in range(all_actions):
        link2[j, j%len(entities)] = 1
    
#     print(link1, link2)
#     print("steps: ", len(steps))
#     print("entities: ", len(entities))
    return link1, link2
    
action[action_step.reversed, action_entity.reversed] = JointSensor(entity[entity_rel], step[context_step], entity['index'], step['index'], forward=make_actions)

def read_labels(*prevs, data):
    print(prevs[0].shape, prevs[1].shape)
    c = data.view(1, -1, *(data.size()[2:]))
    print(c.squeeze(0).shape)
    return c.squeeze(0)
#     pass
    
action[action_label] = FunctionalReaderSensor(action_step.reversed, action_entity.reversed, keyword="Action", forward=read_labels)


In [10]:
from regr.program import POIProgram, IMLProgram, SolverPOIProgram
from regr.program.primaldualprogram import PrimalDualProgram
from regr.program.model.pytorch import SolverModel
from regr.program.metric import MacroAverageTracker, PRF1Tracker, PRF1Tracker, DatanodeCMMetric
from regr.program.loss import NBCrossEntropyLoss, NBCrossEntropyIMLoss

In [11]:
program = SolverPOIProgram(graph, poi=(procedure, action, action_label), inferTypes=['ILP', 'local/argmax'], loss=MacroAverageTracker(NBCrossEntropyLoss()), metric={'ILP':PRF1Tracker(DatanodeCMMetric()),'argmax':PRF1Tracker(DatanodeCMMetric('local/argmax'))})

In [12]:
lbp = program

dataset = ProparaReader(file="data/train.json")  # Adding the info on the reader

#     lbp.test(dataset, device='auto')
all_updates = []
for datanode in lbp.populate(dataset, device="cpu"):
    datanode.inferILPResults(action_label, fun=None)

    final_output = {
        "id": datanode.getAttribute("id"),
        "steps": [],
        "actions": [],
        "steps_before": [],
        "actions_before": [],
    }

    for action_info in datanode.findDatanodes(select=action):
        c = action_info.getAttribute(action_label, "ILP")
        final_output["actions"].append(c)
        c = action_info.getAttribute(action_label)
        final_output["actions_before"].append(c)

    all_updates.append(final_output)

#         print('datanode:', datanode)
#         print('inference spam:', datanode.getAttribute(Spam, 'ILP'))
#         print('inference regular:', datanode.getAttribute(Regular, 'ILP'))

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]]) ['plant; animal', 'soft tissues', 'bones', 'mineral', 'fossils'] [0, 1, 2, 3, 4]
<class 'regr.graph.graph.Graph'>
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])
edgesensor-2
Error list index out of range during updating data item {'Action': tensor([[[3.1940e-01, 4.0114e-01, 1.0121e-02, 2.6934e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01],
         [6.1400e-01, 4.4190e-03, 4.0276e-02, 3.4130e-01]],

        [[7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01],
         [7.0585e-01, 7.2110e-04, 3.8825e-02, 2.5460e-01]],

        [[5.7955e-01, 3.5761e-04, 3.9642e-01, 2.3672e-02],
         [4

IndexError: list index out of range